# Implement different integration schemes

In [1]:
#Lecture 9
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def func(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return a*x**2 + b*x + c

In [3]:
def func_integral(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return (a*x**3)/3. + (b*x**2)/2. + c*x

In [4]:
def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h) + f(x))

In [5]:
def trapezoid_method(f,a,b,N):
    #f == function to integrate
    #a == lower limit of integration
    #b == upper limit of integration
    #N == number of function evalutations to use
    
    #define x values to perform trapezoid rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #peform the integral using the trapezoid method
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
        
    #return the answer
    return Fint

In [6]:
def simpsons_core(f,x,h):
    return h*(f(x) + 4*f(x+h) + f(x+2*h))/3.

In [7]:
def simpsons_method(f,a,b,N):
    #f == function to integrate
    #a == lower limit of integration
    #b == upper limit of integration
    #N == number of intervals to use
    #note the number of chunks will be N-1 so if N is odd, then we don't need to adjust the last segment
    
    #define x values to perform simpsons rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #perform the integral using simpson's method
    for i in range(0,len(x)-2,2):
        Fint += simpsons_core(f,x[i],h)
        
    #apply simpson's rule over the last interval if N is even
    if((N%2)==0):
        Fint += simpsons_core(f,x[-2],0.5*h)
    
    #return the answert
    return Fint

In [8]:
def romberg_core(f,a,b,i):
    
    #we need the differene between a and b
    h = b-a
    
    #interval between function evaluations at refine level i
    dh= h/2.**(i)
    
    #we need the cofactor
    K = h/2.**(i+1)
    
    #and the function evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
        
    #return the answer
    return K*M

In [9]:
def romberg_integration(f,a,b,tol):
    
    #define an interation variable
    i=0
    
    #define a max number of iterations
    imax = 1000
    
    #define an error estimate
    delta = 100.0*np.fabs(tol)
    
    #set an array of integral answers
    I = np.zeros(imax,dtype=float)
    
    #get the zeroth romberg iteration first
    I[0]= 0.5*(b-a)*(f(a) + f(b))
    
    #iterate by 1
    i += 1
    
    #iterate until we reach tolerance
    while(delta>tol):
        
        #find this Romberg iteration
        I[i] = 0.5*I[i-i] + romberg_core(f,a,b,i)
        
        #compute the new fractional error estimate
        delta = np.fabs((I[i]-I[i-1])/I[i])
        
        print(i,I[i],I[i-1],delta)
        
        if(delta>tol):
            
            #iterate
            i+=1
            
            #if we reached the maximum iterations
            if(i>imax):
                print("Max iterations reached.")
                raise StopIteration('Stoppin interations after ',i)
    
    #return the answer
    return I[i]

In [10]:
Answer = func_integral(1)-func_integral(0)
print(Answer)
print("Trapezoid")
print(trapezoid_method(func,0,1,10))
print("Simpson's Method")
print(simpsons_method(func,0,1,10))
print("Romberg")
tolerance = 1.0e-4
RI = romberg_integration(func,0,1,tolerance)
print(RI, (RI-Answer)/Answer, tolerance)

0.8866666666666665
Trapezoid
0.888744855967078
Simpson's Method
0.8866666666666665
Romberg
1 0.9603124999999997 1.0549999999999997 0.09860071591278888
2 0.9682031249999998 0.9603124999999997 0.008149761962398284
3 0.9701757812499998 0.9682031249999998 0.0020332977673987703
4 0.9706689453124997 0.9701757812499998 0.0005080661793925158
5 0.9707922363281247 0.9706689453124997 0.00012700041369444437
6 0.9708230590820311 0.9707922363281247 3.17490953867205e-05
0.9708230590820311 0.09491322452860677 0.0001
